### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 25 ноября 2019, 15:00   
**Штраф за опоздание:** -2 балла после 15:00 25 ноября, -4 балла после 15:00 2 декабря, -6 баллов после 15:00 9 декабря  -8 баллов после 15:00 16 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0919, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) и [Speed Dating Data](https://cloud.mail.ru/public/8nHV/p6J7wY1y1)

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle

class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return np.reshape((1 - np.sum((l_c / l_s) ** 2, axis=1)), (-1, 1)) * l_s + \
                np.reshape((1 - np.sum((r_c / r_s) ** 2, axis=1)), (-1, 1)) * r_s
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return np.reshape(np.sum( - (l_c / l_s) * np.log2(l_c / l_s), axis=1), (-1, 1)) * l_s + \
    np.reshape(np.sum( - (r_c / r_s) * np.log2(r_c / r_s), axis=1), (-1, 1)) * r_s

    def __misclass(self, l_c, l_s, r_c, r_s):
        return np.reshape(1 - np.max(l_c, axis=1) / l_s, (-1, 1)) * l_s + \
                np.reshape(1 - np.max(r_c, axis=1) / r_s, (-1, 1)) * r_s

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[ : int(np.sqrt(n_feature))]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[ : int(np.log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        extra_y = self.min_samples_split // 2
        sorted_x, sorted_y = self.__sort_samples(x, y)
        if extra_y == 0:
            split_y = sorted_y
        else:
            split_y = sorted_y[extra_y : - extra_y]
        y_diff_idx = np.where(split_y[:-1] != split_y[1:])[0] + (extra_y + 1)

        if len(y_diff_idx) == 0:
            return np.NaN, None

        eq_el_count = y_diff_idx - np.append(np.array([extra_y]),
                                               y_diff_idx[:-1])
        one_hot_code = np.zeros((y_diff_idx.shape[0], self.num_class))
        one_hot_code[np.arange(y_diff_idx.shape[0]),
                     sorted_y[y_diff_idx - 1]] = 1
        class_increm = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increm[0] += np.bincount(sorted_y[:extra_y],
                                           minlength=self.num_class)
        l_count = np.cumsum(class_increm, axis=0)
        r_count = np.bincount(sorted_y,
                                    minlength=self.num_class) - l_count
        l_sizes = y_diff_idx.reshape(l_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        g_crit = self.G_function(l_count, l_sizes, r_count, r_sizes)
        idx = np.argmin(g_crit)
        l_idx = l_sizes[idx][0]
        return g_crit[idx], (sorted_x[l_idx - 1] + sorted_x[l_idx]) / 2

    def __fit_node(self, x, y, node_id, depth):
        n_samples = x.shape[0]
        y = y.astype(int)
        maxnum = np.argmax(np.bincount(y))
        if ( self.max_depth == depth ) or ( n_samples <= self.min_samples_split ) or\
           ( self.sufficient_share <= float(maxnum) / n_samples ):
            self.tree[node_id] = (self.LEAF_TYPE, maxnum)
            return
        else:
            n_features = x.shape[1]
            g_crit_vec = np.zeros(n_features).astype(float)
            thresholds = np.zeros(n_features).astype(float)
            for i in self.get_feature_ids(n_features):
                g_crit_vec[i], thresholds[i] = self.__find_threshold(x[:, i], y)
            best_id = np.argmin(g_crit_vec)
            threshold = thresholds[best_id]
            if np.all(self.feature_importances_ is None):
                self.feature_importances_ = np.zeros(n_features).astype(float)
            self.feature_importances_[best_id] = g_crit_vec[best_id] / n_samples

            x_l, x_r, y_l, y_r = self.__div_samples(x, y, best_id, threshold)
            if (x_l.shape[0] == 0 or x_r.shape[0] == 0):
                self.tree[node_id] = (self.LEAF_TYPE, maxnum)
                return

            self.tree[node_id] = (self.NON_LEAF_TYPE, best_id, threshold)
            self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
            self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [9]:
%time clf.fit(X_train, y_train)

Wall time: 2 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

Wall time: 17.9 ms


C:\Users\karry\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in greater


## Проверка качества работы на wine

In [10]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9487179487179486

In [11]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.9474747474747475

## Подготовка данных Speed Dating Data 

In [12]:
df = pd.read_csv('Speed Dating Data.csv', encoding='cp1251')

In [13]:
df = df.iloc[:, :97]

df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)
df = df.drop(['condtn'], axis=1)
df = df.drop(['round'], axis=1)
df = df.drop(['position', 'positin1'], axis=1)
df = df.drop(['order'], axis=1)
df = df.drop(['partner'], axis=1)
df = df.drop(['age_o', 'race_o', 'pf_o_att', 
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o','met_o'], 
             axis=1)
df = df.dropna(subset=['age'])

df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(19)
df = df.drop(['field'], axis=1)

df = df.drop(['undergra'], axis=1)
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)

df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)

df.loc[:, 'race'] = df.loc[:, 'race'].fillna(6)

df = df.dropna(subset=['imprelig', 'imprace'])

df = df.drop(['from', 'zipcode'], axis=1)
df = df.drop(['income'], axis=1)
df = df.dropna(subset=['date'])

df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(18)
df = df.drop(['career'], axis=1)

df = df.drop(['sports','tvsports','exercise', 
              'dining','museums','art','hiking','gaming',
              'clubbing','reading','tv','theater','movies',
              'concerts','music','shopping','yoga'], axis=1)

df = df.drop(['expnum'], axis=1)

df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1',
                                        'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
(df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1',
            'shar1_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100

df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
                                        'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
(df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1',
            'shar2_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)

for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i), 
            'intel{}_1'.format(i), 'fun{}_1'.format(i), 
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    df = df.drop(feat, axis=1)
    
df = df.drop(['wave'], axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1)\
                                 .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                                   .dropna()        
df_female.columns = df_female.columns + '_f'
df_pair = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
d_data = df_pair.iloc[:, 1:].values
d_target = df_pair.iloc[:, 0].values

In [14]:
len(d_target)

3999

## Проверка скорости работы на Speed Dating Data 

In [15]:
X_train, X_test, y_train, y_test = train_test_split(d_data, d_target, test_size=0.33,
                                                    stratify=d_target)
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [16]:
%time clf.fit(X_train, y_train)

Wall time: 92.8 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [17]:
%time my_clf.fit(X_train, y_train)

C:\Users\karry\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in greater


Wall time: 411 ms


## Проверка качества работы на Speed Dating Data

In [18]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5358434415148505

In [19]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.48666666666666664

## Задание 3

In [ ]:
#добавила feature_importances_ в MyDecisionTreeClassifier

## Задание 4

In [20]:
clf = DecisionTreeClassifier(min_samples_split=2)
clf.fit(X_train, y_train)
feature_results = pd.DataFrame({'feature': df_pair.columns[1:],
                                'importance': clf.feature_importances_})
feature_results = feature_results.sort_values('importance',
                                              ascending = False).reset_index(drop=True)
print('Decision Tree Classifier:')
feature_results.head(10)

Decision Tree Classifier:


,feature,importance
0,int_corr,0.066249
1,amb1_1,0.043234
2,fun1_1,0.041983
3,imprelig_f,0.031840
4,date,0.030326
5,imprace_f,0.028711
6,amb2_1_f,0.027945
7,attr2_1,0.026885
8,shar2_1,0.026376
9,fun1_1_f,0.025666


In [21]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
my_clf.fit(X_train, y_train)
feature_results = pd.DataFrame({'feature': df_pair.columns[1:],
                                'importance': my_clf.feature_importances_})
feature_results = feature_results.sort_values('importance',
                                              ascending = False).reset_index(drop=True)
print('My Decision Tree Classifier:')
feature_results.head(10)

C:\Users\karry\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in greater


My Decision Tree Classifier:


,feature,importance
0,sinc2_1_f,0.358209
1,fun1_1_f,0.344916
2,intel2_1,0.323039
3,race_f,0.315151
4,tuition_f,0.307599
5,mn_sat,0.300000
6,go_out_f,0.298667
7,sinc3_1,0.291556
8,career_c_f,0.289855
9,exphappy,0.284582


## Задание 5

In [24]:
param = {'criterion': ['gini', 'entropy'],
         'min_samples_split': range(2, 10), 'n_estimators': range(5, 15)}
srch = GridSearchCV(RandomForestClassifier(), param, cv=3)
srch.fit(X_train, y_train)
srch.best_params_

{'criterion': 'gini', 'min_samples_split': 9, 'n_estimators': 11}

In [23]:
param = {'criterion': ['gini', 'entropy'],
         'min_samples_split': range(2, 10), 'n_estimators': range(5, 15)}
search = RandomizedSearchCV(RandomForestClassifier(), param, cv=3)
search.fit(X_train, y_train)
search.best_params_

{'n_estimators': 11, 'min_samples_split': 4, 'criterion': 'gini'}